In [63]:
import pprint 
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from psycopg2.extensions import AsIs 
import os

def pg_con(): 
    con = psycopg2.connect(
            dbname='postgres', 
            user='admin', 
            host='localhost', 
            port='30960', 
            password='password'
    )
    return(con) 

In [64]:
pg_con()

<connection object at 0x105e56ac8; dsn: 'user=admin password=xxx dbname=postgres host=localhost port=30960', closed: 0>

In [65]:
c=pg_con().cursor() 

In [66]:
c

<cursor object at 0x105ba0dd8; closed: 0>

In [74]:
try:
    c.execute("CREATE TABLE IF NOT EXISTS test2 (id integer, qa text);")
except:
    c.execute("rollback")
    c.execute("CREATE TABLE IF NOT EXISTS test2 (id integer, qa text);")

In [ ]:
c.execute("INSERT INTO test2 VALUES(1,'abc')")

In [88]:
c.execute("select * from test2")
c.fetchall()

[(1, 'abc')]

In [89]:
import pandas as pd
df = pd.read_csv('/Users/jasminekaur/Desktop/basf/Code/data/Boston_Crime_Dataset.csv',encoding="latin_1",dtype='unicode' )  

In [90]:
df.dtypes

INCIDENT_NUMBER        object
OFFENSE_CODE           object
OFFENSE_CODE_GROUP     object
OFFENSE_DESCRIPTION    object
DISTRICT               object
REPORTING_AREA         object
OCCURRED_ON_DATE       object
YEAR                   object
MONTH                  object
DAY_OF_WEEK            object
HOUR                   object
UCR_PART               object
STREET                 object
Lat                    object
Long                   object
Location               object
dtype: object

In [99]:
df_OFFENSE = df[['OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION']].dropna()
df_OFFENSE['OFFENSE_DETAILS']=df_OFFENSE['OFFENSE_CODE']+'_'+df_OFFENSE['OFFENSE_CODE_GROUP']+'_'+df_OFFENSE['OFFENSE_DESCRIPTION']
df_OFFENSE['OFFENSE_DETAILS'] = df_OFFENSE['OFFENSE_DETAILS'].str.replace(' ', '')

In [100]:
df_OFFENSE.nunique()

OFFENSE_CODE           222
OFFENSE_CODE_GROUP      97
OFFENSE_DESCRIPTION    262
OFFENSE_DETAILS        302
dtype: int64

In [101]:
df_OFFENSE = df_OFFENSE.drop_duplicates(subset='OFFENSE_DETAILS', keep='first') #.reset_index()
df_OFFENSE['OFFENSE_ID']=range(1, len(df_OFFENSE.index)+1)
df_OFFENSE = df_OFFENSE[['OFFENSE_ID','OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION','OFFENSE_DETAILS']]
df_OFFENSE=df_OFFENSE.astype('str')

In [102]:
df_OFFENSE.isnull().sum()

OFFENSE_ID             0
OFFENSE_CODE           0
OFFENSE_CODE_GROUP     0
OFFENSE_DESCRIPTION    0
OFFENSE_DETAILS        0
dtype: int64

In [103]:
df_OFFENSE.head()

,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,OFFENSE_DETAILS
0,1,2403,Disorderly Conduct,DISTURBING THE PEACE,2403_DisorderlyConduct_DISTURBINGTHEPEACE
1,2,3201,Property Lost,PROPERTY - LOST,3201_PropertyLost_PROPERTY-LOST
2,3,2647,Other,THREATS TO DO BODILY HARM,2647_Other_THREATSTODOBODILYHARM
3,4,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,413_AggravatedAssault_ASSAULT-AGGRAVATED-BATTERY
4,5,3122,Aircraft,AIRCRAFT INCIDENTS,3122_Aircraft_AIRCRAFTINCIDENTS


KeyError: "['OFFENSE_ID', 'LOCATION_ID'] not in index"

In [104]:
df=df.astype('str')

In [107]:
c.execute("""CREATE TABLE INCIDENT(
    INCIDENT_NUMBER text,
    OFFENSE_CODE integer,
    OFFENSE_CODE_GROUP text,
    OFFENSE_DESCRIPTION text,
    DISTRICT text, 
    REPORTING_AREA integer,
    OCCURRED_ON_DATE integer,
    YEAR integer,
    MONTH integer,
    DAY_OF_WEEK integer,
    HOUR integer,
    UCR_PART integer,
    STREET text,
    Lat float,
    Long float,
    LOCATION text);""")
con.commit()

DuplicateTable: relation "incident" already exists


In [97]:
Incident_Cols = "`,`".join([str(i) for i in df.columns.tolist()])
Incident_Cols

'INCIDENT_NUMBER`,`OFFENSE_CODE`,`OFFENSE_CODE_GROUP`,`OFFENSE_DESCRIPTION`,`DISTRICT`,`REPORTING_AREA`,`OCCURRED_ON_DATE`,`YEAR`,`MONTH`,`DAY_OF_WEEK`,`HOUR`,`UCR_PART`,`STREET`,`Lat`,`Long`,`Location'